In [ ]:
# Import libraries
import pandas as pd
# pd.plotting.register_matplotlib_converters()
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import re
from datetime import datetime
from sklearn.ensemble import IsolationForest
from statsmodels.graphics import tsaplots
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from fbprophet import Prophet
from pylab import rcParams
import calendar

In [ ]:
# Read data
df = pd.read_csv('train_1.csv/train_1.csv')
df.head()

In [ ]:
# Fill missing values
df = df.fillna(0)

In [ ]:
# Reshaping the dataframe
df_reshaped = pd.melt(df,id_vars=['Page'],var_name='Date',value_name='Views')
df_reshaped.head()

In [ ]:
# Setting 'Date' column as the index 
df_reshaped['Date'] = pd.to_datetime(df_reshaped['Date']) 
df_reshaped = df_reshaped.set_index('Date')

## Time series analysis

In [ ]:
# Average number of views per day
temp = df_reshaped.groupby('Date')['Views'].mean()
plt.figure(figsize=(15,4))
plt.xlabel('Date')
plt.ylabel('Avg views')
plt.title('Average number of views per day')
plt.plot(temp,label='Views')
plt.legend()
plt.show()

In [ ]:
# Average number of views per month
month_index = df_reshaped.index.month
views_by_month = df_reshaped.groupby(month_index).mean()
months=['January','February','March','April','May','June','July','August','September','October','November','December']
ax = views_by_month.plot()
start, end = ax.get_xlim()
plt.xticks(np.arange(start+0.5, end+0.5, 1.0))
ax.set_xticklabels(months,rotation=75)
ax.set_xlabel('Month')
ax.set_ylabel('Average views')
ax.set_title('Average number of views per month')
plt.show()

In [ ]:
weekday_index = df_reshaped.index.weekday_name
views_by_weekdays = df_reshaped.groupby(weekday_index).sum()

ax = views_by_weekdays.plot()
#start, end = ax.get_xlim()
#plt.xticks(np.arange(start+0.5, end+0.5, 1.0))
#ax.set_xticklabels(months,rotation=75)
#ax.set_xlabel('Month')
ax.set_ylabel('Total views')
plt.show()

In [ ]:
def detect_lang(page):
    text = page.split('.wikipedia')
    if re.search('[a-z][a-z]',text[0][-2:]):
        return text[0][-2:]
    else: 
        return 'none'
temp1 = df_reshaped  
temp1['Wikipedia_page'] = temp1.Page.apply(detect_lang)

def lang_code(code):
    if code == 'zh':
        return 'Chinese'
    elif code == 'fr':
        return 'French'
    elif code == 'en':
        return 'English'
    elif code == 'ru':
        return 'Russian'
    elif code == 'de':
        return 'German'
    elif code == 'ja':
        return 'Japanese'
    elif code == 'es':
        return 'Spanish'
    else:
        return 'None'
    
temp1['Page_language'] = temp1.Wikipedia_page.apply(lang_code)

In [ ]:
# Total number of views based on language of Wikipedia webpage
fig,ax = plt.subplots(figsize=(15,6))
lang_df = temp1.groupby('Page_language')['Views'].sum().reset_index()
lang_df.head()
# lang_df = lang_df[lang_df['Page_language']!='None']
# # lang_df['Views'] = round(lang_df['Views']/1000000,0)

# bar_graph = lang_df.plot.bar(x='Page_language',y='Views',rot=30,ax=ax)
# bar_graph.set_ylabel('Total views (in millions)')
# bar_graph.set_title('Total number of views based on language of webpage')

# for p in ax.patches:
#     ax.annotate(str(p.get_height()), (p.get_x()+0.1, p.get_height()+500))
# ax.legend()
# plt.show()

In [ ]:
df_reshaped['Dayofweek'] = df_reshaped.Date.dt.dayofweek

def find_day(day):
    if day == 0:
        return 'Monday'
    elif day == 1:
        return 'Tuesday'
    elif day == 2:
        return 'Wednesday'
    elif day == 3:
        return 'Thursday'
    elif day == 4:
        return 'Friday'
    elif day == 5:
        return 'Saturday'
    else:
        return 'Sunday'
    
df_reshaped['Dayofweek'] = df_reshaped.Dayofweek.apply(find_day)

In [ ]:
day = df_reshaped.groupby('Dayofweek')['Views'].sum()
plt.figure(figsize=(15,4))
plt.xlabel('Day of week')
plt.ylabel('Total views')
plt.plot(day)
plt.show()

In [ ]:
top_pages = df_reshaped.groupby('Page')['Views'].sum().reset_index()
top_pages.nlargest(5,'Views')

In [ ]:
top5_pages_df = df_reshaped[df_reshaped['Page'].isin(['Main_Page_en.wikipedia.org_all-access_all-agents','Main_Page_en.wikipedia.org_desktop_all-agents','Main_Page_en.wikipedia.org_mobile-web_all-agents','Wikipedia:Hauptseite_de.wikipedia.org_all-access_all-agents','Special:Search_en.wikipedia.org_all-access_all-agents'])]

In [ ]:
del pd

In [ ]:
for i in list(top5_pages_df.Page.unique())[:5]:
    ax = top5_pages_df.loc[top5_pages_df.Page == i,:].plot(label=i)
ax.set_ylabel('Views')
plt.legend(loc='upper left')
plt.show()

In [ ]:
top5_pages_df.head()

In [ ]:
table = pd.pivot_table(top5_pages_df,values='Views',index=['Date'],columns=['Page'])

In [ ]:
corr_matrix = table.corr(method='pearson')
sns.clustermap(corr_matrix)

In [ ]:
top_page_df = df_reshaped[df_reshaped.Page == 'Main_Page_en.wikipedia.org_all-access_all-agents']
top_page_df = top_page_df[['Views']]
top_page_df['Views'] = top_page_df['Views'].div(1000000).round(2)
top_page_df.head()

In [ ]:
ax1 = top_page_df.boxplot()

In [ ]:
ax2 = top_page_df.plot(kind='density', linewidth=2)
ax2.set_xlabel('Views')
plt.show()

## Anomaly detection using Isolation Forest

In [ ]:
isolation_forest_model = IsolationForest(contamination=0.08)
isolation_forest_model.fit(top_page_df)
top_page_df['anomaly'] = isolation_forest_model.predict(top_page_df)

In [ ]:
anomaly_df.head()

In [ ]:
anomaly_df.reset_index().info()

In [ ]:
fig, ax = plt.subplots(figsize=(11,9))
anomaly_df = top_page_df.loc[top_page_df['anomaly'] == -1].copy()
ax.plot(top_page_df.index,top_page_df['Views'],color='blue', label = 'Normal')
plt.scatter(anomaly_df.index,anomaly_df['Views'],color='red',s=100,label = 'Anomaly')
plt.legend()
plt.show()

In [ ]:
top_page_df = top_page_df[top_page_df['anomaly']==1]
top_page_df.drop(columns=['anomaly'],inplace=True)

In [ ]:
plt.figure(figsize=(11,9))
plt.plot(top_page_df,label='Views')
plt.title('Data after removing anomalies')
plt.show()

In [ ]:
fig = tsaplots.plot_acf(top_page_df['Views'], lags=60)
plt.show()

In [ ]:
rcParams['figure.figsize'] = 11,9
ts_decomposition = sm.tsa.seasonal_decompose(top_page_df['Views'],freq = 30)
figure = ts_decomposition.plot()
plt.show()

## Augmented Dicky-Fuller test

In [ ]:
test = adfuller(top_page_df['Views'])
print(test)

 0th element is the test statistic. Since the value is approximately -3, we can say that the data is more likely to be stationary.
 1st element indicates the p-value. Since the p-value is < 0.05, we can reject null hypothesis.

In [ ]:
ax = top_page_df.plot()
plt.show()

In [ ]:
top_page_filtered = top_page_df[['Views']]
stationary_df = top_page_filtered.diff().dropna()
stationary_df.head()

In [ ]:
ax = stationary_df.plot()
plt.show()

In [ ]:
test2 = adfuller(stationary_df['Views'])
print(test2)

In [ ]:
fig = tsaplots.plot_acf(stationary_df['Views'], lags=14)
plt.show()

## Modelling

In [ ]:
train = top_page_filtered[:'2016-09']
test = top_page_filtered['2016-10':]

In [ ]:
fig = tsaplots.plot_acf(train['Views'], lags=30)
plt.show()

In [ ]:
fig = tsaplots.plot_pacf(train['Views'], lags=30)
plt.show()

In [ ]:
aic_bic_values = []
for p in range(7):
    for q in range(7):
        try:
            model = SARIMAX(train, order=(p,0,q))
            result = model.fit()
            aic_bic_values.append((p,q,result.aic,result.bic))
            print(p, q, result.aic, result.bic)
        except:
            print(p,q,None,None)


In [ ]:
aic_bic_df = pd.DataFrame(aic_bic_values,columns=['p','q','aic','bic'])
print(aic_bic_df.sort_values('aic'))

## Fitting ARMA model

In [ ]:
model = SARIMAX(train,order=(6,0,2))
result = model.fit()

In [ ]:
forecast = result.get_prediction(start=-30)
forecast_mean = forecast.predicted_mean

In [ ]:
forecast_mean.head()

In [ ]:
confidence_interval = forecast.conf_int()
confidence_interval.head()

## Predicting the number of views for the last 30 days using ARMA model

In [ ]:
fig,ax=plt.subplots()
train[-30:].rename(columns={'Views':'actual value'}).plot(ax=ax)
forecast_mean.plot(ax=ax,label='prediction')
plt.fill_between(confidence_interval.index, \
                confidence_interval['lower Views'], \
                confidence_interval['upper Views'], \
                color='pink', alpha=0.5)
plt.legend()
plt.show()

### Forecasting using ARMA model

In [ ]:
forecast_values = result.get_forecast(steps=test.shape[0])
forecast_values_mean = forecast_values.predicted_mean
conf_interval = forecast_values.conf_int()

In [ ]:
fig,ax=plt.subplots()
top_page_filtered.rename(columns={'Views':'Actual value'}).plot(ax=ax)
forecast_values_mean.plot(ax=ax,label='Forecast')
plt.fill_between(conf_interval.index, \
                conf_interval['lower Views'], \
                conf_interval['upper Views'], \
                color='pink', alpha=0.5)
plt.title('Forecasted number of views for the next 30 days')
plt.legend()
plt.show()

In [ ]:
print('RMSE: '+str(np.sqrt(np.mean(np.square(forecast_values_mean.values - test.Views.values)))))

## ARIMA model

In [ ]:
arima_aic_bic = []
for p in range(7):
    for q in range(7):
        try:
            arima_model = SARIMAX(train, order=(p,1,q))
            arima_result = arima_model.fit()
            arima_aic_bic.append((p,q,arima_result.aic,arima_result.bic))
        except:
            continue

arima_aic_bic_df = pd.DataFrame(arima_aic_bic,columns=['p','q','aic','bic'])
print(arima_aic_bic_df.sort_values('aic'))

In [ ]:
arima_model = SARIMAX(train,order=(5,1,6))
arima_result = arima_model.fit()
arima_forecast_mean = arima_result.get_prediction(steps=-30).predicted_mean

In [ ]:
arima_result.plot_diagnostics()
plt.show()

In [ ]:
arima_result.summary()

In [ ]:
fig,ax=plt.subplots()
train[-30:].rename(columns={'Views':'actual value'}).plot(ax=ax)
arima_forecast_mean[-30:].plot(ax=ax,label='prediction')
plt.legend()
plt.show()

### Forecasting using ARIMA model

In [ ]:
arima_forecast_values = arima_result.get_forecast(steps=test.shape[0])
arima_forecast_mean = arima_forecast_values.predicted_mean
arima_conf_interval = arima_forecast_values.conf_int()

In [ ]:
fig,ax=plt.subplots()
top_page_filtered.rename(columns={'Views':'Actual value'}).plot(ax=ax)
arima_forecast_mean.plot(ax=ax,label='Forecast')
plt.fill_between(arima_conf_interval.index, \
                arima_conf_interval['lower Views'], \
                arima_conf_interval['upper Views'], \
                color='pink', alpha=0.5)
plt.title('Forecasted number of views for the next 30 days')
plt.legend()
plt.show()

In [ ]:
print('RMSE: '+str(np.sqrt(np.mean(np.square(arima_forecast_mean.values - test.Views.values)))))

## Exponential smoothing model

In [ ]:
exp_smoothing_model = SimpleExpSmoothing(train)
exp_smoothing_result = exp_smoothing_model.fit(smoothing_level=0.5,optimized=True)
exp_smoothing_prediction = exp_smoothing_result.predict(start=train[-30:-29].index[0],end=train[-1:].index[0])
plt.plot(train[-30:], label='Train')
plt.plot(exp_smoothing_prediction, label='Prediction')
plt.legend(loc='best')
plt.show()

### Forecasting using Exponential Smoothing model

In [ ]:
exp_smoothing_forecast = exp_smoothing_result.forecast(test.shape[0])
plt.plot(top_page_filtered)
plt.plot(exp_smoothing_forecast, label='Forecast')
plt.legend(loc='best')
plt.show()

In [ ]:
print('RMSE: '+str(np.sqrt(np.mean(np.square(exp_smoothing_forecast.values - test.Views.values)))))

## Prophet

In [ ]:
ptrain = b[:'2016-09']
ptest = b['2016-10':]
prophet_model = Prophet()
prophet_result = prophet_model.fit(ptrain.reset_index().rename(columns={'Date':'ds','Views':'y'}))

In [ ]:
future = prophet_model.make_future_dataframe(periods=ptest.shape[0])
forecast = prophet_model.predict(future)

In [ ]:
train.shape[0]

In [ ]:
prophet_forecast = forecast[['ds','yhat_lower','yhat_upper','yhat']][-ptest.shape[0]:]
plt.plot(prophet_forecast['ds'],prophet_forecast['yhat'],label='forecast')
plt.plot(b, label='Actual data')
# plt.fill_between(prophet_forecast['ds'],prophet_forecast['yhat_lower'],prophet_forecast['yhat_upper'])
plt.legend(loc='best')
plt.show()

In [ ]:
fig = prophet_model.plot_components(forecast)

In [ ]:
fig1 = prophet_model.plot(forecast)